# Tom and Alex Violet Project 2
# April 13, 2025



In [9]:
import CollabFilterOneVectorPerItem
from AbstractBaseCollabFilterSGD import AbstractBaseCollabFilterSGD

from CollabFilterOneVectorPerItem import CollabFilterOneVectorPerItem
from train_valid_test_loader import load_train_valid_test_datasets


# Import plotting libraries
import matplotlib
import matplotlib.pyplot as plt

In [10]:
# # Step 3i: no regularization (alpha = 0) and K values (n_factors) 2, 10, and 50
# from CollabFilterOneVectorPerItem import CollabFilterOneVectorPerItem
# from train_valid_test_loader import load_train_valid_test_datasets
# # K value array for training models
# K = [2, 10, 50]

# # Load Data
# train_tuple, valid_tuple, test_tuple, n_users, n_items = load_train_valid_test_datasets()

# # Train model
# for i in K:  
#     print("K value: ", i)
    
#     model = CollabFilterOneVectorPerItem(
#         n_epochs=10, 
#         batch_size=1000, 
#         step_size=0.1,
#         n_factors=i,
#         alpha=0.0)

#     model.init_parameter_dict(n_users, n_items, train_tuple)

#     # Fit the model with SGD
#     model.fit(train_tuple, valid_tuple)
#     # print("Valid Set --- RMSE: ", model.rmse, "MAE " , model.mae)
#     # RMSE and MAE on the validation set
#     # RMSE and MAE on the test set
#     # value of parameters μ,b,c,u,v (not needed for your report, but useful so you don't need to retrain the model if you have to remake a figure)

In [11]:
K = [2, 10, 50]
colors = ['b', 'g', 'm']
# Load Data
train_tuple, valid_tuple, test_tuple, n_users, n_items = load_train_valid_test_datasets()
all_traces = []
# Train model
for i in K:  
    print("K value: ", i)
    
    model = CollabFilterOneVectorPerItem(
        n_epochs=10, 
        batch_size=1000, 
        step_size=0.1,
        n_factors=i,
        alpha=500)

    model.init_parameter_dict(n_users, n_items, train_tuple)

    # Fit the model with SGD
    model.fit(train_tuple, valid_tuple)
    all_traces.append((K, model.trace_epoch, model.trace_rmse_train, model.trace_rmse_valid))

K value:  2
epoch       0.000 | loss_total     0.00432 | tr RMSE     1.24455 | va RMSE     1.24854 | grad_wrt_mu     0.00095 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00081 | grad_wrt_V     0.00082
epoch       0.014 | loss_total     0.00376 | tr RMSE     1.24451 | va RMSE     1.24850 | grad_wrt_mu     0.00110 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00073 | grad_wrt_V     0.00074
epoch       0.029 | loss_total     0.00346 | tr RMSE     1.24446 | va RMSE     1.24845 | grad_wrt_mu     0.00124 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00066 | grad_wrt_V     0.00066
epoch       0.043 | loss_total     0.00303 | tr RMSE     1.24441 | va RMSE     1.24839 | grad_wrt_mu     0.00106 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00059 | grad_wrt_V     0.00060
epoch       0.129 | loss_total     0.00199 | tr RMSE     1.24414

epoch       0.386 | loss_total     0.00160 | tr RMSE     1.24335 | va RMSE     1.24731 | grad_wrt_mu     0.00105 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00005 | grad_wrt_V     0.00005
epoch       0.500 | loss_total     0.00152 | tr RMSE     1.24299 | va RMSE     1.24694 | grad_wrt_mu     0.00106 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00002 | grad_wrt_V     0.00002
epoch       0.629 | loss_total     0.00154 | tr RMSE     1.24257 | va RMSE     1.24651 | grad_wrt_mu     0.00103 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00001 | grad_wrt_V     0.00001
epoch       0.757 | loss_total     0.00155 | tr RMSE     1.24217 | va RMSE     1.24610 | grad_wrt_mu     0.00115 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00000 | grad_wrt_V     0.00000
epoch       0.886 | loss_total     0.00143 | tr RMSE     1.24175 | va RMSE  

epoch       0.757 | loss_total     0.00149 | tr RMSE     1.24216 | va RMSE     1.24609 | grad_wrt_mu     0.00104 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00000 | grad_wrt_V     0.00000
epoch       0.886 | loss_total     0.00145 | tr RMSE     1.24176 | va RMSE     1.24568 | grad_wrt_mu     0.00104 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00000 | grad_wrt_V     0.00000
epoch       1.000 | loss_total     0.00147 | tr RMSE     1.24140 | va RMSE     1.24531 | grad_wrt_mu     0.00096 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00000 | grad_wrt_V     0.00000
epoch       1.129 | loss_total     0.00151 | tr RMSE     1.24101 | va RMSE     1.24491 | grad_wrt_mu     0.00113 | grad_wrt_b_per_user     0.00000 | grad_wrt_c_per_item     0.00000 | grad_wrt_U     0.00000 | grad_wrt_V     0.00000
epoch       1.257 | loss_total     0.00160 | tr RMSE     1.24059 | va RMSE  

In [ ]:
for idx, (K, epochs, rmse_train, rmse_valid) in enumerate(all_traces):
    plt.plot(epochs, rmse_train, f'{colors[idx]}.-', label=f'Train RMSE (K={K})')
    plt.plot(epochs, rmse_valid, f'{colors[idx]}--', label=f'Valid RMSE (K={K})')

plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.legend(loc='lower right')
plt.ylim([0, 3])
plt.title('SGD Training RMSE (α=0)')
plt.show()